## Il Fenomeno del "Tunneling" in Unity

**Cos'è:**
Il Tunneling è un glitch fisico comune dove un oggetto che si muove velocemente attraversa un ostacolo solido (come un muro) senza rilevare la collisione, pur avendo un Collider e un Rigidbody attivi.

**Perché succede (La Teoria):**
Unity non calcola la fisica in modo "continuo" come nella realtà, ma a intervalli discreti (default: ogni 0.02 secondi). Immagina la fisica come una serie di fotografie scattate in sequenza:

1. **Frame A:** Il Player è 1 metro *davanti* al muro.
2. **Frame B:** Il Player si muove ad alta velocità. Nel frame successivo si trova già 1 metro *oltre* il muro.

Se il motore fisico controlla solo la posizione nelle "foto" (modalità *Discrete*), vede che in nessun momento il Player ha toccato il muro. Risultato: il Player passa attraverso come un fantasma.

### Come risolvere: Le 4 Strategie

Se stai usando correttamente `Rigidbody.MovePosition` o applicando velocità nel `FixedUpdate` e il problema persiste, la causa è quasi sempre la configurazione. Ecco le soluzioni in ordine di priorità:

**1. Collision Detection: Continuous (La soluzione standard)**
Questa è la prima cosa da fare per oggetti veloci (Player, Proiettili).

* **Dove:** Sul `Rigidbody` dell'oggetto che si muove (non sul muro!).
* **Azione:** Cambia `Collision Detection` da **Discrete** a **Continuous**.
* **Cosa fa:** Invece di teletrasportare l'oggetto tra due frame, Unity calcola una "scia" (sweep) volumetrica tra la vecchia e la nuova posizione. Se la scia tocca il muro, la collisione viene rilevata.

**2. Geometria e Spessori (Design)**
Se i muri sono troppo sottili (es. un `Plane` o un cubo molto schiacciato), è facile che il movimento in un singolo frame superi interamente lo spessore del muro.

* **Soluzione:** Aumenta lo spessore dei Collider dei muri (proprietà `Size` del BoxCollider) o usa oggetti più grossi. Un muro spesso è più difficile da "saltare".

**3. Fixed Timestep (Precisione Globale)**
Se il gioco è estremamente veloce, il controllo ogni 0.02s potrebbe essere troppo lento.

* **Dove:** `Edit > Project Settings > Time`.
* **Azione:** Abbassa il `Fixed Timestep` (es. da `0.02` a `0.01`).
* **Pro/Contro:** Raddoppia la precisione della fisica, ma raddoppia anche il carico sulla CPU. Da usare con cautela.

**4. Predictive Raycasting (Soluzione via Codice)**
Se le soluzioni sopra falliscono (casi estremi), bisogna prevenire il movimento via script.

* **Logica:** Prima di muovere il Rigidbody, lancia un raggio (`Physics.Raycast`) dalla posizione attuale verso la direzione del movimento, con lunghezza pari alla velocità per il tempo del frame.
* **Risultato:** Se il raggio tocca qualcosa, muovi l'oggetto solo fino al punto di impatto, ignorando la velocità residua.

---
